In [1]:
import os
import sys

from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
#from langchain.vectorstores import Milvus
#from langchain.vectorstores import OpenAIVectorStore

In [2]:
import urllib.request

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="06_07_test.pdf")

('06_07_test.pdf', <http.client.HTTPMessage at 0x104a20910>)

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader( "06_07_test.pdf" )
pages = loader.load_and_split()


In [5]:
print( type(loader), type(pages), type(pages[0]), type(pages[0].page_content))
print(  sys.getsizeof(loader), len(pages), sys.getsizeof(pages[0]), len(pages[0].page_content) )

<class 'langchain_community.document_loaders.pdf.PyPDFLoader'> <class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>
48 445 72 57


In [6]:
for i in range( 5 ):
    print( "pages[", i, "]=", len(pages[i].page_content), )

chunks = [ pages.page_content for pages in pages ]
print( len(chunks), type(chunks) )
print( 'Chunk Max:', max(len(achunk) for achunk in chunks ) )
print( 'Chunk Min:', min(len(achunk) for achunk in chunks ) )
print( 'sumChunks:', sum(len(achunk) for achunk in chunks ) )
print( '#of chunk:', len(chunks) )
print( 'Chunk Avg:', sum(len(achunk) for achunk in chunks ) / len(chunks) )
print( 'Chunk Avg:', sum(map( len, chunks )) / len(chunks) )

#print( pages[0] ) # 또는 print( pages[0] )
#print( chunks[0] ) # 또는 print( chunks[0] )

pages[ 0 ]= 57
pages[ 1 ]= 60
pages[ 2 ]= 53
pages[ 3 ]= 512
pages[ 4 ]= 468
445 <class 'list'>
Chunk Max: 1640
Chunk Min: 6
sumChunks: 372229
#of chunk: 445
Chunk Avg: 836.4696629213483
Chunk Avg: 836.4696629213483


In [5]:
print( pages[0] )
print( chunks[0] )

page_content='2023
북한인권보고서
2023 Report on North Korean Human Rights
통일부' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-31T13:50:27+09:00', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False', 'source': '06_07_test.pdf', 'total_pages': 448, 'page': 0, 'page_label': '1'}
2023
북한인권보고서
2023 Report on North Korean Human Rights
통일부


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    chunk_overlap=0,
)

splitedDocs = textSplitter.split_documents( pages )
# 48 496 72 57 < 1000
# 48 445 72 57 < 2000
# 48 445 72 57 < 3000

In [7]:
print( type(textSplitter), type(splitedDocs), type(splitedDocs[0]), type(splitedDocs[0].page_content) )
print( sys.getsizeof(textSplitter), len(splitedDocs), sys.getsizeof(splitedDocs[0]), len(splitedDocs[0].page_content) )

<class 'langchain_text_splitters.character.RecursiveCharacterTextSplitter'> <class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>
48 496 72 57


In [8]:
for i in range( 5 ):
    print( "splitedDocs[", i, "]=", len(splitedDocs[i].page_content), )

chunks = [ splitedDocs.page_content for splitedDocs in splitedDocs ]
print( len(chunks) )
print( 'Chunk Max:', max(len(achunk) for achunk in chunks ) )
print( 'Chunk Min:', min(len(achunk) for achunk in chunks ) )
print( 'sumChunks:', sum(len(achunk) for achunk in chunks ) )
print( '#of chunk:', len(chunks) )
print( 'Chunk Avg:', sum(len(achunk) for achunk in chunks ) / len(chunks) )
print( 'Chunk Avg:', sum(map( len, chunks )) / len(chunks) )

print( splitedDocs[0] )
print( chunks[0] )

splitedDocs[ 0 ]= 57
splitedDocs[ 1 ]= 60
splitedDocs[ 2 ]= 53
splitedDocs[ 3 ]= 512
splitedDocs[ 4 ]= 468
496
Chunk Max: 1000
Chunk Min: 6
sumChunks: 372148
#of chunk: 496
Chunk Avg: 750.2983870967741
Chunk Avg: 750.2983870967741
page_content='2023
북한인권보고서
2023 Report on North Korean Human Rights
통일부' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-31T13:50:27+09:00', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False', 'source': '06_07_test.pdf', 'total_pages': 448, 'page': 0, 'page_label': '1'}
2023
북한인권보고서
2023 Report on North Korean Human Rights
통일부


In [12]:
from langchain.embeddings import OpenAIEmbeddings
#from langchain.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


def process_in_batches(documents, batch_size=100):
    embeddings = OpenAIEmbeddings()
    vdbx = None

    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        if vdbx is None:
            vdbx = Chroma.from_documents(batch, embeddings)
        else:
            vdbx.add_documents(batch)
        print(f"처리된 문서: {i + len(batch)}/{len(documents)}")
    return vdbx

vdb =
vdb = process_in_batches(splitedDocs)
print("vdb적재문서수=", vdb._collection.count())


처리된 문서: 100/496
처리된 문서: 200/496
처리된 문서: 300/496
처리된 문서: 400/496
처리된 문서: 496/496
vdb적재문서수= 992


In [ ]:
print( len( vdb ), type( vdb ) )

In [2]:
## exception sample

try:
    import chromadb
except Exception as e:
    print("CHROMADB LOAD ERROR:", repr(e))


try:
    import chromadb
except Exception as e:
    import traceback
    traceback.print_exc()